<a href="https://colab.research.google.com/github/Trangquyh/12102003/blob/master/aspect_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Cài đặt các thư viện cần thiết
!pip install transformers torch pyvi scikit-learn pandas emoji

import pandas as pd
import numpy as np
import torch
import re
import emoji
from pyvi import ViTokenizer
from torch.utils.data import Dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, pipeline)
from sklearn.model_selection import train_test_split

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang chạy trên: {device}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 93.4 MB/s eta 0:00:00
Đang chạy trên: cuda


In [3]:
# 1. Đọc file Excel đã gán nhãn
try:
    df = pd.read_excel("/content/train_data_feedbacks.xlsx", sheet_name="feedbacks")
    print(">>> Đã load file thành công!")
except:
    print(">>> LỖI: Bạn chưa upload file 'train_data.xlsx' hoặc sai tên file.")

# 2. Kiểm tra và làm sạch dữ liệu
# Đảm bảo cột feedback là string và không bị rỗng
df = df.dropna(subset=['comments'])
df['comments'] = df['comments'].astype(str)


>>> Đã load file thành công!


In [4]:
target_cols = ['safety', 'smell', 'texture', 'spec', 'logistics']

# Chuyển đổi các cột này sang dạng số
for col in target_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# Tạo cột 'labels' chứa vector, ví dụ: [1, 0, 0, 0, 1]
df['labels'] = df[target_cols].values.tolist()

# Hiển thị 5 dòng đầu để kiểm tra
print("\n>>> Dữ liệu sau khi xử lý:")
display(df[['comments', 'labels']].head())

# In ra số lượng mẫu
print(f"\n>>> Tổng số lượng mẫu train: {len(df)}")


>>> Dữ liệu sau khi xử lý:


,comments,labels
0,"Giấy khá chắc dai, mềm mịn hoa văn rất đẹp giá...","[0, 0, 1, 0, 1]"
1,"Giao hàng nhanh, đóng gói cẩn thận, giấy dùng ...","[0, 0, 0, 0, 1]"
2,"Giao hàng siêu nhanh, hnay đặt mai giao tới lu...","[0, 0, 1, 1, 1]"
3,"Giấy của shop mình thì khỏi bàn cãi, gđ mình 4...","[0, 0, 0, 1, 0]"
4,"Mình thích sản phẩm giấy bên shop , vừa dai , ...","[0, 0, 1, 0, 1]"



>>> Tổng số lượng mẫu train: 296


# Fine-tuning Model

In [14]:
# 5. Tokenize cho PhoBERT (Tách từ: "giao hàng" -> "giao_hàng")
df['phobert_input'] = df['comments'].apply(lambda x: ViTokenizer.tokenize(x))

# Kiểm tra dữ liệu trước khi train
print("\n>>> Dữ liệu sẵn sàng:")
display(df[['comments', 'phobert_input', 'labels']].head())


>>> Dữ liệu sẵn sàng:


,comments,phobert_input,labels
0,"giấy khá chắc dai, mềm mịn hoa văn rất đẹp giá...","giấy khá chắc dai , mềm mịn hoa_văn rất đẹp gi...","[0, 0, 1, 0, 1]"
1,"giao hàng nhanh, đóng gói cẩn thận, giấy dùng ...","giao hàng nhanh , đóng_gói cẩn_thận , giấy dùn...","[0, 0, 0, 0, 1]"
2,"giao hàng siêu nhanh, hôm nay đặt mai giao tới...","giao hàng siêu nhanh , hôm_nay đặt mai giao tớ...","[0, 0, 1, 1, 1]"
3,"giấy của cửa hàng mình thì khỏi bàn cãi, gđ mì...","giấy của cửa_hàng mình thì khỏi bàn_cãi , gđ m...","[0, 0, 0, 1, 0]"
4,"mình thích sản phẩm giấy bên cửa hàng , vừa da...","mình thích sản_phẩm giấy bên cửa_hàng , vừa da...","[0, 0, 1, 0, 1]"


In [15]:
aspect_label = target_cols

In [16]:
# Cấu hình
MODEL_NAME = "vinai/phobert-base"
MAX_LEN = 200 # Độ dài câu tối đa

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Dataset Class
class HygieneDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.df.iloc[index]['phobert_input']
        labels = self.df.iloc[index]['labels']

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            # Quan trọng: Multi-label classification yêu cầu kiểu FloatTensor
            'labels': torch.tensor(labels, dtype=torch.float)
        }

# Chia tập Train (80%) và Validation (20%)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = HygieneDataset(train_df, tokenizer, MAX_LEN)
val_dataset = HygieneDataset(val_df, tokenizer, MAX_LEN)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [17]:
# Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(aspect_label),
    problem_type="multi_label_classification"
)

# Hàm tính độ chính xác (Custom Metrics)
def compute_metrics(pred):
    # Lấy giá trị dự đoán (logits)
    predictions = pred.predictions
    # Dùng hàm Sigmoid để chuyển về xác suất 0-1
    probs = 1 / (1 + np.exp(-predictions))
    # Ngưỡng cut-off là 0.5
    preds = (probs > 0.5).astype(int)
    labels = pred.label_ids

    # Tính Accuracy: Chỉ tính là đúng nếu đoán đúng toàn bộ vector
    # VD: Nhãn [1,0,0] mà đoán [1,1,0] -> Sai
    exact_match_acc = (preds == labels).all(axis=1).mean()

    # Tính F1-micro (Thường dùng cho multi-label để đánh giá tốt hơn)
    # Tuy nhiên để đơn giản ta dùng Accuracy từng nhãn
    return {'accuracy': exact_match_acc}

# Cấu hình Training
training_args = TrainingArguments(
    output_dir='./results_hygiene',
    num_train_epochs=15,             # Tăng lên 15 epochs vì dữ liệu ít (400 dòng)
    per_device_train_batch_size=8,   # Batch size nhỏ để ổn định
    per_device_eval_batch_size=16,
    learning_rate=2e-5,              # Tốc độ học chuẩn cho BERT
    weight_decay=0.01,
    eval_strategy="epoch",     # Kiểm tra sau mỗi vòng lặp
    save_strategy="no",              # Không lưu checkpoint rác để tiết kiệm ổ cứng
    logging_steps=10
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# BẮT ĐẦU TRAIN
print(">>> Bắt đầu huấn luyện...")
trainer.train()

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

RobertaForSequenceClassification LOAD REPORT from: vinai/phobert-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.decoder.bias            | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
roberta.pooler.dense.bias       | UNEXPECTED | 
roberta.pooler.dense.weight     | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.decoder.weight          | UNEXPECTED | 
classifier.out_proj.weight      | MISSING    | 
classifier.dense.bias           | MISSING    | 
classifier.out_proj.bias        | MISSING    | 
classifier.dense.weight         | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initia

>>> Bắt đầu huấn luyện...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.603293,0.557457,0.033333
2,0.530853,0.534075,0.200000
3,0.484649,0.486687,0.350000
4,0.431388,0.439146,0.316667
5,0.385966,0.410233,0.400000
6,0.361133,0.390588,0.466667
7,0.326537,0.376347,0.433333
8,0.284274,0.368119,0.466667
9,0.257067,0.348683,0.450000
10,0.233548,0.342203,0.433333


TrainOutput(global_step=450, training_loss=0.33600361320707534, metrics={'train_runtime': 132.8524, 'train_samples_per_second': 26.646, 'train_steps_per_second': 3.387, 'total_flos': 363843056376000.0, 'train_loss': 0.33600361320707534, 'epoch': 15.0})

In [46]:
# 1. Lưu model
save_path = "./my_hygiene_aspect_model"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)
print(f">>> Model đã được lưu tại: {save_path}")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

>>> Model đã được lưu tại: ./my_hygiene_aspect_model


In [44]:
import pandas as pd
import torch
import re
import emoji
from pyvi import ViTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import os # Import os module

class CustomerFeedbackAnalyzer:
    def __init__(self, aspect_model_path, sentiment_model_name="wonrax/phobert-base-vietnamese-sentiment"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Đang khởi tạo Analyzer trên thiết bị: {self.device}")

        # --- Initialize dictionaries first, as they are needed for preprocessing ---
        self._init_dictionaries()

        # --- 1. LOAD MODEL ASPECT (Đã train) ---
        print(f"... Đang load Aspect Model từ: {aspect_model_path}")
        if not os.path.isdir(aspect_model_path): # Check if the model path exists
            print(f"❌ Lỗi: Đường dẫn model '{aspect_model_path}' không tồn tại hoặc không phải thư mục. Vui lòng kiểm tra lại.")
            self.aspect_tokenizer = None
            self.aspect_model = None
        else:
            try:
                self.aspect_tokenizer = AutoTokenizer.from_pretrained(aspect_model_path, local_files_only=True)
                self.aspect_model = AutoModelForSequenceClassification.from_pretrained(aspect_model_path, local_files_only=True)
                self.aspect_model.to(self.device)
                self.aspect_model.eval()
            except Exception as e:
                print(f"❌ Lỗi load Aspect Model: {e}")
                print("Hãy đảm bảo bạn đã chạy lệnh 'trainer.save_model()' và đường dẫn chính xác và chứa đủ các file model.")
                self.aspect_tokenizer = None # Indicate failure
                self.aspect_model = None # Indicate failure

        self.aspect_labels = ['safety', 'smell', 'texture', 'spec', 'logistics']

        # --- 2. LOAD MODEL SENTIMENT (HuggingFace) ---
        print(f"... Đang load Sentiment Model: {sentiment_model_name}")
        self.sentiment_task = pipeline(
            "sentiment-analysis",
            model=sentiment_model_name,
            tokenizer=sentiment_model_name,
            device=0 if torch.cuda.is_available() else -1)

    def _init_dictionaries(self):
        """Khởi tạo các từ điển dùng cho preprocessing và sub-issue"""
        self.SUB_ISSUE_KEYWORDS = {
            "SPEC": {
                "Thiếu hàng/Số lượng": ["thiếu", "không đủ", "đếm", "sót", "ít hơn", "hụt", "đủ gói", "không đủ số lượng"],
                "Kích thước/Độ dày": ["bé", "nhỏ", "mỏng", "ngắn", "size", "dày", "khổ", "to", "kích thước"],
                "Màu sắc/Mẫu mã": ["màu", "xấu", "hình", "mẫu", "đen", "ố", "vàng", "trắng", "mẫu mã", "hình ảnh"],
                "Bao bì (Thẩm mỹ)": ["bao bì", "vỏ", "nắp", "decal", "tem", "xinh", "đẹp"]
            },
            "TEXTURE": {
                "Bụi giấy": ["bụi", "vụn", "hắt xì", "bay", "lông", "xơ"],
                "Độ dai/bền": ["bở", "mủn", "rách", "nát", "dính da"],
                "Độ mềm": ["ráp", "cứng", "đau", "khô"]
            },
            "SMELL": {
                "Mùi Tre/Gỗ": ["tre", "trúc", "gỗ"],
                "Mùi Hóa chất": ["hắc", "cồn", "tẩy", "nồng", "nhức đầu"],
                "Mùi Mốc/Hư hỏng": ["mốc", "hôi", "chua", "thối"]
            },
            "LOGISTICS": {
                "Sai mẫu mã": ["sai mẫu", "nhầm", "không đúng loại", "đặt cái này giao cái kia", "lấy nhầm"],
                "Thiếu hàng": ["thiếu", "không đủ", "đếm", "sót", "ít hơn"],
                "Hư hỏng vật lý": ["móp", "bẹp", "rách", "nát", "ướt", "thủng", "dúm"],
                "Thời gian giao": ["chậm", "lâu", "trễ", "đợi", "mãi", "nhanh"],
                "Dịch vụ/Thái độ": ["thái độ", "không trả lời", "tư vấn", "chửi", "lồi lõm", "nhiệt tình"]
            },
            "SAFETY": {
                "Kích ứng da": ["ngứa", "đỏ", "dị ứng", "hại", "độc", "đau", "rát"]
            }
        }

        self.teencode_dict = {
            "k": "không", "ko": "không", "kh": "không", "hok": "không", "hông": "không", "kg": "không",
            "ch": "chưa", "chura": "chưa",
            "dc": "được", "đk": "được", "dk": "được", "đc": "được",
            "bt": "bình thường", "bth": "bình thường", "bthg": "bình thường",
            "ah": "à", "bik": "biết", "hỉu": "hiểu", "nch": "nói chung",
            "f": "ph", "đt":"điện thoại",
            "r": "rồi", "dầy": "dày", "hnay": "hôm nay", "TQ": "Trung Quốc", "VN": "Việt Nam",
            "đc": "được", "gd": "gia đình", "tg": "thời gian", "nma": "nhưng mà",

            "ship": "giao hàng", "shiper": "giao hàng", "shipper": "giao hàng",
            "shop": "cửa hàng", "sop": "cửa hàng", "xop": "cửa hàng", "xốp": "cửa hàng",
            "rep": "trả lời", "ib": "nhắn tin", "nt": "nhắn tin",
            "fb": "đánh giá", "feedback": "đánh giá",
            "tks": "cảm ơn", "thank": "cảm ơn", "thanks": "cảm ơn", "vote": "đánh giá", "gởi": "gửi", "thg": "thùng",

            "sp": "sản phẩm", "spham": "sản phẩm",
            "gvs": "giấy vệ sinh",
            "sz": "kích thước", "size": "kích thước",
            "sl": "số lượng", "cl": "chất lượng",


            "ok": "tốt", "oke": "tốt", "oki": "tốt", "okie": "tốt", "okela": "tốt",
            "good": "tốt", "gut": "tốt", "bad": "tệ",
            "wa": "quá", "qa": "quá",
            "nh": "nhiều", "nhiu": "nhiều", "nhìu": "nhiều",
            "r": "rồi", "roi": "rồi", "rùi": "rồi",
            "sd": "sử dụng", "thik": "thích",
            'qá': "quá", "wá": "quá",
            'vs': 'với', 'j': "gì",'m': "mình", "mik": "mình",'h': "giờ"
        }

        self.emoji_dict = {
            "👍": " tốt ",
            "10₫": "tuyệt vời",
            "❤": " yêu thích ", "♥": " yêu thích ", "♡": " yêu thích ",
            "😍": " thích ", "🥰": " thích ", "😘": " thích ",
            "🤩": " tuyệt vời ",
            "👌": " tốt ", "👏": " hoan hô ",
            "💯": " tuyệt đối ", "⭐": " sao ",
            "🌹": " hoa ", "💐": " hoa ",
            "🤣": " cười ", "😂": " cười ", "😆": " cười ",

            "😡": " tức giận ", "😠": " tức giận ", "🤬": " chửi thề ",
            "👎": " tệ ",
            "☹": " buồn ", "😞": " thất vọng ", "😔": " thất vọng ",
            "😭": " khóc ", "bad": " tệ ",
            "😤": " bực mình ",

            "🤢": " buồn nôn ", "🤮": " buồn nôn ",
            "😷": " khẩu trang ",
            "🤧": " hắt xì ",
            "🤕": " đau ", "hea": " đau ",

            "🙄": " ngán ngẩm ", "😑": " cạn lời ",
            "🤔": " suy nghĩ ",
            "🙏": " cảm ơn "
        }

        self.dict_char_map = {
            'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ',
            'òe': 'oè', 'óe': 'oé', 'ỏe': 'oẻ', 'õe': 'oẽ', 'ọe': 'oẹ',
            'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ', 'ụy': 'uỵ'
        }

    def normalize_text(self, text):
        if not isinstance(text, str): return ""
        text = text.lower()

        for icon, word in self.emoji_dict.items():
            if icon in text: text = text.replace(icon, word)

        for old, new in self.dict_char_map.items():
            text = text.replace(old, new)

        text = re.sub(r'aw', 'ă', text)
        text = re.sub(r'ow', 'ơ', text)
        text = re.sub(r'uw', 'ư', text)
        text = re.sub(r'dd', 'đ', text)

        text = re.sub(r'([a-zđăâêôơưáàảãạấầẩẫậắằẳẵặéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ])\1+', r'\1', text)

        pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in self.teencode_dict.keys()) + r')\b')
        text = pattern.sub(lambda x: self.teencode_dict[x.group()], text)

        text = re.sub(r'[^\w\s\.\,\?\!\;]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    def get_sub_issue(self, text, aspect):
        if aspect not in self.SUB_ISSUE_KEYWORDS: return "General/Other"

        text = text.lower()

        for sub_label, keywords in self.SUB_ISSUE_KEYWORDS[aspect].items():
            for kw in keywords:
                if kw in text:
                    return sub_label

        return "General/Other"

    def analyze_single_feedback(self, raw_text):
        clean_text = self.normalize_text(raw_text)
        tokenized_text = ViTokenizer.tokenize(clean_text)

        detected_aspects = []
        if self.aspect_model is not None and self.aspect_tokenizer is not None:
            inputs = self.aspect_tokenizer(tokenized_text, return_tensors="pt", truncation=True, max_length=128).to(self.device)
            with torch.no_grad():
                outputs = self.aspect_model(**inputs)
            probs = torch.nn.Sigmoid()(outputs.logits).cpu().numpy()[0]

            for i, label in enumerate(self.aspect_labels):
                threshold = 0.2 if label == 'safety' else 0.5
                if probs[i] > threshold:
                    detected_aspects.append(label)
        else:
            print(f"Warning: Aspect model not loaded for text: {raw_text}. Skipping aspect prediction.")

        input_sentiment = " ".join(clean_text.split()[:200])
        sent_result = self.sentiment_task(input_sentiment)[0]
        sentiment_label = {'NEG': 'Negative', 'POS': 'Positive', 'NEU': 'Neutral'}.get(sent_result['label'], 'Neutral')
        sentiment_score = sent_result['score']

        results = []
        if not detected_aspects:
            results.append({
                "raw_text": raw_text,
                "clean_text": clean_text,
                "aspect": "General/Other",
                "sub_issue": "N/A",
                "sentiment": sentiment_label,
                "sentiment_score": sentiment_score
            })
        else:
            for aspect in detected_aspects:
                aspect_upper = aspect.upper()
                sub_issue = self.get_sub_issue(clean_text, aspect_upper)
                results.append({
                    "raw_text": raw_text,
                    "clean_text": clean_text,
                    "aspect": aspect_upper,
                    "sub_issue": sub_issue,
                    "sentiment": sentiment_label,
                    "sentiment_score": sentiment_score
                })
        return results

    def process_file(self, input_file_path, output_file_path="final_report.xlsx", text_col="comments"):
        """
        Hàm xử lý trọn gói từ File -> File
        """
        if input_file_path.endswith('.csv'):
            df = pd.read_csv(input_file_path)
        else:
            df = pd.read_excel(input_file_path)

        print(f">>> Bắt đầu xử lý {len(df)} dòng dữ liệu...")

        final_rows = []
        for index, row in df.iterrows():
            if index % 10 == 0: print(f"Processing row {index}/{len(df)}...")

            try:
                feedback_content = str(row[text_col])
                analysis_results = self.analyze_single_feedback(feedback_content)
                final_rows.extend(analysis_results)
            except Exception as e:
                print(f"Lỗi tại dòng {index}: {e}")

        result_df = pd.DataFrame(final_rows)

        result_df.to_excel(output_file_path, index=False)
        print(f"✅ Đã hoàn tất! File kết quả lưu tại: {output_file_path}")
        return result_df

In [47]:
# --- BƯỚC 1: Cấu hình đường dẫn ---
# Đường dẫn đến folder model bạn đã save bằng lệnh trainer.save_model()
# Trong file colab cũ của bạn là "./my_hygiene_aspect_model"
model_path = "./my_hygiene_aspect_model"

# --- BƯỚC 2: Khởi tạo Analyzer (Chạy 1 lần) ---
analyzer = CustomerFeedbackAnalyzer(aspect_model_path=model_path)

# --- BƯỚC 3: Chạy trên file dữ liệu mới ---
# Upload file Excel chứa feedback chưa gán nhãn lên Colab (ví dụ: new_data.xlsx)
input_file = "/content/train_data_feedbacks.xlsx" # Thay tên file của bạn vào đây

# Gọi hàm xử lý trọn gói
# text_col="comments": Thay "comments" bằng tên cột chứa nội dung feedback trong file của bạn
df_result = analyzer.process_file(
    input_file_path=input_file,
    output_file_path="Bao_cao_phan_tich_final.xlsx",
    text_col="comments"
)

# Xem trước 5 dòng kết quả
display(df_result.head())

Đang khởi tạo Analyzer trên thiết bị: cuda
... Đang load Aspect Model từ: ./my_hygiene_aspect_model


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

... Đang load Sentiment Model: wonrax/phobert-base-vietnamese-sentiment


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: wonrax/phobert-base-vietnamese-sentiment
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


>>> Bắt đầu xử lý 297 dòng dữ liệu...
Processing row 0/297...
Processing row 10/297...
Processing row 20/297...
Processing row 30/297...
Processing row 40/297...
Processing row 50/297...
Processing row 60/297...
Processing row 70/297...
Processing row 80/297...
Processing row 90/297...
Processing row 100/297...
Processing row 110/297...
Processing row 120/297...
Processing row 130/297...
Processing row 140/297...
Processing row 150/297...
Processing row 160/297...
Processing row 170/297...
Processing row 180/297...
Processing row 190/297...
Processing row 200/297...
Processing row 210/297...
Processing row 220/297...
Processing row 230/297...
Processing row 240/297...
Processing row 250/297...
Processing row 260/297...
Processing row 270/297...
Processing row 280/297...
Processing row 290/297...
✅ Đã hoàn tất! File kết quả lưu tại: Bao_cao_phan_tich_final.xlsx


,raw_text,clean_text,aspect,sub_issue,sentiment,sentiment_score
0,"Giấy khá chắc dai, mềm mịn hoa văn rất đẹp giá...","giấy khá chắc dai, mềm mịn hoa văn rất đẹp giá...",TEXTURE,Độ mềm,Positive,0.992931
1,"Giấy khá chắc dai, mềm mịn hoa văn rất đẹp giá...","giấy khá chắc dai, mềm mịn hoa văn rất đẹp giá...",LOGISTICS,General/Other,Positive,0.992931
2,"Giao hàng nhanh, đóng gói cẩn thận, giấy dùng ...","giao hàng nhanh, đóng gói cẩn thận, giấy dùng ...",LOGISTICS,Thời gian giao,Positive,0.992950
3,"Giao hàng siêu nhanh, hnay đặt mai giao tới lu...","giao hàng siêu nhanh, hôm nay đặt mai giao tới...",TEXTURE,General/Other,Positive,0.992897
4,"Giao hàng siêu nhanh, hnay đặt mai giao tới lu...","giao hàng siêu nhanh, hôm nay đặt mai giao tới...",SPEC,Kích thước/Độ dày,Positive,0.992897
